In [65]:
class BaseComponent:
    high_sent: int
    low_sent: int
    key: str
    destinations: list[str]

    def __init__(self, key):
        self.key = key
        self.high_sent = 0
        self.low_sent = 0
        self.destinations = []

    def send(self, pulse) -> list[tuple[str, str, int]]:
        instructions = list(map(lambda x: (self.key, x, pulse), self.destinations))
        for i in instructions:
            if i[2] == 0:
                self.low_sent = self.low_sent + 1
            else:
                self.high_sent = self.high_sent + 1
        return instructions

    def get_state(self):
        return None

    def process(self, pulse, from_key) -> list[tuple[str, str, int]]:
        result = self.handle_pulse(pulse, from_key)
        if result is not None:
            return self.send(result)

        return []

    def handle_pulse(self, pulse, from_key) -> "int | None":
        # Abstract to be implemented
        pass

    def reset(self) -> None:
        pass


class Output(BaseComponent):
    low_recieved: int
    color = "yellow"

    def __init__(self, key):
        super().__init__(key)
        self.low_recieved = 0

    def handle_pulse(self, pulse, from_key) -> "int | None":
        if pulse == 0:
            self.low_recieved = self.low_recieved + 1
        return None


class Broadcaster(BaseComponent):
    color = "green"

    def __init__(self, key: str):
        super().__init__(key)

    def handle_pulse(self, pulse, _) -> "int | None":
        # Does nothing to the pulse, sends it on to others
        return pulse


class FlipFlop(BaseComponent):
    color = "red"
    
    active: bool

    def __init__(self, key: str):
        super().__init__(key)
        self.active = False

    def reset(self) -> None:
        self.active = False

    def get_state(self):
        return self.active

    def process(self, pulse, _) -> list[tuple[str, str, int]]:
        # print(f"FLIP FLOP RECIEVED - {pulse}")
        if pulse == 1:
            # print("HIGH")
            return []
        else:
            # print("LOW")
            self.active = not self.active
            if self.active:
                return self.send(1)
            else:
                return self.send(0)


class Conjunction(BaseComponent):
    color = "purple"

    input_state: dict[str, int]

    def __init__(self, key):
        super().__init__(key)
        self.input_state = {}

    def get_state(self):
        return self.input_state

    def reset(self) -> None:
        for key in self.input_state.keys():
            self.input_state[key] = 0

    def handle_pulse(self, pulse, from_key) -> "int | None":
        self.input_state[from_key] = pulse
        if 0 in self.input_state.values():
            return 1

        return 0


def print_instructions(ins):
    for i in ins:
        print(f"{i[0]} -{'high' if i[2] == 1 else 'low'}-> {i[1]}")


def push_button(comps):
    processed = []
    instructions = [("button", "broadcaster", 0)]
    while len(instructions) > 0:
        processed.append(instructions[0])
        from_comp, to_comp, pulse = instructions[0]
        result = comps[to_comp].process(pulse, from_comp)

        instructions = [*instructions[1:], *result]
    
    return processed

def get_mapped_to(val, mapps):
    result = []
    for m in mapps:
        if m[1] == val:
            result.append(m[0])
    return result


In [66]:
mappings = []
components = {}
comps = {}

with open("input.txt", "r", encoding="utf-8") as file:
    for line in file.readlines():
        from_comp, to_comps = line.split("->")

        from_comp_name = from_comp.strip()
        if from_comp_name == "broadcaster":
            components[from_comp_name] = from_comp_name
            from_comp_name_parsed = from_comp_name
        else:
            components[from_comp_name[1:]] = from_comp_name[0]
            from_comp_name_parsed = from_comp_name[1:]

        for comp in to_comps.strip().split(","):
            if comp not in components:
                components[comp] = "no"

            mappings.append((from_comp_name_parsed, comp.strip()))

for key, val in components.items():
    if val == "broadcaster":
        comps[key] = Broadcaster(key)
    elif val == "%":
        comps[key] = FlipFlop(key)
    elif val == "&":
        comps[key] = Conjunction(key)
    else:
        comps[key] = Output(key)

for mapping in mappings:
    comps[mapping[0]].destinations.append(mapping[1])

    if components[mapping[1]] == "&":
        comps[mapping[1]].input_state[mapping[0]] = 0

pushes = 1000
for i in range(0, pushes):
    push_button(comps)
    # instructions = [("button", "broadcaster", 0)]
    # while len(instructions) > 0:
    #     # print_instructions([instructions[0]])
    #     from_comp, to_comp, pulse = instructions[0]
    #     result = comps[to_comp].process(pulse, from_comp)

    #     # print(f"{from_comp} {to_comp} {pulse} created {result}")

    #     instructions = [*instructions[1:], *result]

print("Done")

total_high = 0
total_low = pushes
for c in comps.values():
    total_high = total_high + c.high_sent
    total_low = total_low + c.low_sent

print(total_high)
print(total_low)

print("======== PART 1 ===========")
print(total_low * total_high)

print("======== PART 2 ===========")
for c in comps.values():
    c.reset()


next_comps = ["rx"]
seen = ["rx"]
while len(next_comps) > 0:
    next_up = []
    for n in next_comps:
        results = get_mapped_to(n, mappings)
        next_up.extend(results)

    next_up = list(set(next_up))
    next_up = list(filter(lambda x: x not in seen, next_up))
    seen.extend(next_up)
    next_comps = next_up


import networkx as nx
import matplotlib.pyplot as plt

graph = nx.DiGraph()

ignorable = []
ignorable = list(filter(lambda x: x not in seen, comps.keys()))
for ignore in ignorable:
    comps.pop(ignore)

for comp in comps.values():
    comp.destinations = list(
        filter(lambda x: x not in ignorable, comp.destinations)
    )

labels = {}
colors = {}
for comp in comps.keys():
    graph.add_node(comp, label=comp)
    colors[comp] = comps[comp].color
    labels[comp] = comp

for mapping in mappings:
    graph.add_edge(mapping[0], mapping[1])

print(graph.nodes())

pos = nx.spring_layout(graph)
for i in pos:
    pos[i][0] = pos[i][0] * 4 # x coordinate
    pos[i][1] = pos[i][1] * 4 # y coordinate

plt.figure(3,figsize=(12,12)) 
nx.draw(graph, pos, labels=labels)

Done
42647
16687
======== PART 1 ===========
711650489
======== PART 2 ===========


TypeError: 'str' object does not support item assignment